<a href="https://colab.research.google.com/github/oburckard/PFE/blob/master/PFE_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from time import time


# Constantes
N = 20
g = 9.8 #gravite, 9,8m/s

# Valeurs des pas
dx = 1.0/(N+1.0) 
dy = 1.0/(N+1.0)
dt = 0.002 #dt<min(dx,dy)
dt2 = dt*dt
dx2 = dx*dx
dy2 = dy*dy
dt2_dx2 = dt2/dx2
dt2_dy2 = dt2/dy2


xi = np.arange(0, 1, dx, dtype='float') 
yj = np.arange(0, 1, dy, dtype='float')
t = np.arange(0, 1, dt, dtype='float') 




#Initialisation des matrices
F_nm1 = np.zeros((N+1,N+1)) #matrice au temps n-1
F_n = np.zeros((N+1,N+1))  #matrice au temps n
F_np1 = np.zeros((N+1,N+1)) #matrice au temps n+1


#On appelle h la profondeur 
#definition de la topographie
def h(x,y): #on prend une profondeur valant 1 partout
  if 0.2<x<0.8 and 0.2<y<0.8 :
      return 1.0
  else :
      return 2.0

def fonc_source_nm1(x,y):
  return np.exp(-100*((x-0.5)**2+(y-0.5)**2))

def fonc_source_n(x,y):  
  return np.exp(-100*((x-0.5)**2+(y-0.5)**2))

vfonc_nm1 = np.vectorize(fonc_source_nm1)
vfonc_nm1(xi, yj)
#Fonction source aux 2 premiers temps
#for i in range(0,N+1):
 # for j in range(0,N+1):
      #F_nm1[i,j]=np.exp(-10*(xi[i]**2+yj[j]**2))#*np.sin(np.pi*t[tp]) #debut de la vague dans un coin
  #    F_nm1[i,j]=np.exp(-100*((xi[i]-0.5)**2+(yj[j]-0.5)**2))#*np.sin(np.pi*t[tp]) #debut au centre
   #   F_n[i,j]=F_nm1[i,j]



#Representation graphique
ite = 0
X, Y = np.meshgrid(xi, yj)
for tp in t:
  ite += 1

  
  #Pour determiner la position a l'instant suivant
  F_np1 = -F_nm1 + 2*F_n
  for i in range(1,N,1):
    for j in range(1,N,1):
      F_np1[i,j] = F_np1[i,j] - g*h(xi[i],yj[j]) * ((2*F_n[i,j]-F_n[i-1,j]-F_n[i+1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j-1]-F_n[i,j+1])*dt2_dy2 )
      F_np1[i,j] += g*( (F_n[i+1,j] - F_n[i-1,j]) *(h(xi[i+1],yj[j]) - h(xi[i-1],yj[j])) / (4.0*dx2)  )
      F_np1[i,j] += g*( (F_n[i,j+1] - F_n[i,j-1]) *(h(xi[i],yj[j+1]) - h(xi[i],yj[j-1])) / (4.0*dy2)  )
      

  #Conditions aux limites (vaut 0 quand on "passe" le bord)

  #i=j=0
  i=0
  j=0
  F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j] - g*h(xi[i],yj[j]) * ( (2*F_n[i,j]-F_n[i+1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j+1])*dt2_dy2 )
  F_np1[i,j] += g * F_n[i+1,j] * h(xi[i+1],yj[j]) / (4.0*dx2) 
  F_np1[i,j] += g * F_n[i,j+1] * h(xi[i],yj[j+1]) / (4.0*dy2) 
      
  #F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j]*(1-g*h(xi[i],yj[j])*(dt2_dx2+dt2_dy2))
  #F_np1[i,j] = F_np1[i,j]+F_n[i+1,j]*(g*h(xi[i],yj[j])*dt2_dx2+g*dt2_dx2*0.25*h(xi[i+1],yj[j]))
  #F_np1[i,j] = F_np1[i,j]+F_n[i,j+1]*(g*h(xi[i],yj[j])*dt2_dy2+g*dt2_dy2*0.25*h(xi[i],yj[j+1]))

  #i=0, 0<j<N
  for j in range(1,N,1):
    F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j] - g*h(xi[i],yj[j]) * ((2*F_n[i,j]-F_n[i+1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j-1]-F_n[i,j+1])*dt2_dy2 )
    F_np1[i,j] += g * F_n[i+1,j] * h(xi[i+1],yj[j]) / (4.0*dx2)
    F_np1[i,j] += g*( (F_n[i,j+1] - F_n[i,j-1]) *(h(xi[i],yj[j+1]) - h(xi[i],yj[j-1])) / (4.0*dy2)  )
      
    #F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j]*(1-g*h(xi[i],yj[j])*(dt2_dx2+dt2_dy2))
    #F_np1[i,j] = F_np1[i,j]+F_n[i+1,j]*(g*h(xi[i],yj[j])*dt2_dx2+g*dt2_dx2*0.25*h(xi[i+1],yj[j]))
    #F_np1[i,j] = F_np1[i,j]+F_n[i,j+1]*(g*h(xi[i],yj[j])*dt2_dy2+g*dt2_dy2*0.25*(h(xi[i],yj[j+1])-h(xi[i],yj[j-1])))+F_n[i,j-1]*(g*h(xi[i],yj[j])*dt2_dy2-g*dt2_dy2*0.25*(h(xi[i],yj[j+1])-h(xi[i],yj[j-1])))
  
  #i=0, j=N
  j=N

  F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j] - g*h(xi[i],yj[j]) * ((2*F_n[i,j]-F_n[i+1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j-1])*dt2_dy2 )
  F_np1[i,j] += g* F_n[i+1,j] * h(xi[i+1],yj[j]) / (4.0*dx2)  
  F_np1[i,j] += g*( - F_n[i,j-1]) *(- h(xi[i],yj[j-1])) / (4.0*dy2)  
      
  #F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j]*(1-g*h(xi[i],yj[j])*(dt2_dx2+dt2_dy2))
  #F_np1[i,j] = F_np1[i,j]+F_n[i+1,j]*(g*h(xi[i],yj[j])*dt2_dx2+g*dt2_dx2*0.25*h(xi[i+1],yj[j]))
  #F_np1[i,j] = F_np1[i,j]+F_n[i,j-1]*(g*h(xi[i],yj[j])*dt2_dy2-g*dt2_dy2*0.25*(-h(xi[i],yj[j-1])))
  
  #0<i<N, j=0
  j=0
  for i in range(1,N,1):
    F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j] - g*h(xi[i],yj[j]) * ((2*F_n[i,j]-F_n[i-1,j]-F_n[i+1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j+1])*dt2_dy2 )
    F_np1[i,j] += g*( (F_n[i+1,j] - F_n[i-1,j]) *(h(xi[i+1],yj[j]) - h(xi[i-1],yj[j])) / (4.0*dx2)  )
    F_np1[i,j] += g* F_n[i,j+1] *h(xi[i],yj[j+1]) / (4.0*dy2)  
      
    #F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j]*(1-g*h(xi[i],yj[j])*(dt2_dx2+dt2_dy2))
    #F_np1[i,j] = F_np1[i,j]+F_n[i+1,j]*(g*h(xi[i],yj[j])*dt2_dx2+g*dt2_dx2*0.25*(h(xi[i+1],yj[j])-h(xi[i-1],yj[j])))+F_n[i-1,j]*(g*h(xi[i],yj[j])*dt2_dx2-g*dt2_dx2*0.25*(h(xi[i+1],yj[j])-h(xi[i-1],yj[j])))
    #F_np1[i,j] = F_np1[i,j]+F_n[i,j+1]*(g*h(xi[i],yj[j])*dt2_dy2+g*dt2_dy2*0.25*h(xi[i],yj[j+1]))

  #i=N, j=0
  i=N
  F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j] - g*h(xi[i],yj[j]) * ((2*F_n[i,j]-F_n[i-1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j+1])*dt2_dy2 )
  F_np1[i,j] += g*(  -F_n[i-1,j]) *( - h(xi[i-1],yj[j])) / (4.0*dx2)  
  F_np1[i,j] += g* F_n[i,j+1]  *h(xi[i],yj[j+1])  / (4.0*dy2)  
      
  #F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j]*(1-g*h(xi[i],yj[j])*(dt2_dx2+dt2_dy2))
  #F_np1[i,j] = F_np1[i,j]+F_n[i-1,j]*(g*h(xi[i],yj[j])*dt2_dx2-g*dt2_dx2*0.25*(-h(xi[i-1],yj[j])))
  #F_np1[i,j] = F_np1[i,j]+F_n[i,j+1]*(g*h(xi[i],yj[j])*dt2_dy2+g*dt2_dy2*0.25*(h(xi[i],yj[j+1])))
  
  #i=N, 1<j<N
  for j in range(1,N,1):
    F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j] - g*h(xi[i],yj[j]) * ((2*F_n[i,j]-F_n[i-1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j-1]-F_n[i,j+1])*dt2_dy2 )
    F_np1[i,j] += g*(  - F_n[i-1,j]) *( - h(xi[i-1],yj[j])) / (4.0*dx2)  
    F_np1[i,j] += g*( (F_n[i,j+1] - F_n[i,j-1]) *(h(xi[i],yj[j+1]) - h(xi[i],yj[j-1])) / (4.0*dy2)  )
      
    #F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j]*(1-g*h(xi[i],yj[j])*(dt2_dx2+dt2_dy2))
    #F_np1[i,j] = F_np1[i,j]+F_n[i-1,j]*(g*h(xi[i],yj[j])*dt2_dx2-g*dt2_dx2*0.25*(-h(xi[i-1],yj[j])))
    #F_np1[i,j] = F_np1[i,j]+F_n[i,j+1]*(g*h(xi[i],yj[j])*dt2_dy2+g*dt2_dy2*0.25*(h(xi[i],yj[j+1])-h(xi[i],yj[j-1])))+F_n[i,j-1]*(g*h(xi[i],yj[j])*dt2_dy2-g*dt2_dy2*0.25*(h(xi[i],yj[j+1])-h(xi[i],yj[j-1]))) 
  
  #i=j=N
  j=N
  F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j] - g*h(xi[i],yj[j]) * ((2*F_n[i,j]-F_n[i-1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j-1])*dt2_dy2 )
  F_np1[i,j] += g*( ( - F_n[i-1,j]) *(- h(xi[i-1],yj[j])) / (4.0*dx2)  )
  F_np1[i,j] += g*( (- F_n[i,j-1]) *( - h(xi[i],yj[j-1])) / (4.0*dy2)  )
      
  #F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j]*(1-g*h(xi[i],yj[j])*(dt2_dx2+dt2_dy2))
  #F_np1[i,j] = F_np1[i,j]+F_n[i-1,j]*(g*h(xi[i],yj[j])*dt2_dx2-g*dt2_dx2*0.25*(-h(xi[i-1],yj[j])))
  #F_np1[i,j] = F_np1[i,j]+F_n[i,j-1]*(g*h(xi[i],yj[j])*dt2_dy2-g*dt2_dy2*0.25*(-h(xi[i],yj[j-1])))

  #0<i<N, j=N
  for i in range(1,N,1):
    F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j] - g*h(xi[i],yj[j]) * ((2*F_n[i,j]-F_n[i-1,j]-F_n[i+1,j])*dt2_dx2 + (2*F_n[i,j]-F_n[i,j-1])*dt2_dy2 )
    F_np1[i,j] += g*( (F_n[i+1,j] - F_n[i-1,j]) *(h(xi[i+1],yj[j]) - h(xi[i-1],yj[j])) / (4.0*dx2)  )
    F_np1[i,j] += g*( (- F_n[i,j-1]) *( - h(xi[i],yj[j-1])) / (4.0*dy2)  )
      
    #F_np1[i,j] = -F_nm1[i,j]+2*F_n[i,j]*(1-g*h(xi[i],yj[j])*(dt2_dx2+dt2_dy2))
    #F_np1[i,j] = F_np1[i,j]+F_n[i+1,j]*(g*h(xi[i],yj[j])*dt2_dx2+g*dt2_dx2*0.25*(h(xi[i+1],yj[j])-h(xi[i-1],yj[j])))+F_n[i-1,j]*(g*h(xi[i],yj[j])*dt2_dx2-g*dt2_dx2*0.25*(h(xi[i+1],yj[j])-h(xi[i-1],yj[j])))
    #F_np1[i,j] = F_np1[i,j]+F_n[i,j-1]*(g*h(xi[i],yj[j])*dt2_dy2-g*dt2_dy2*0.25*(-h(xi[i],yj[j-1])))

 
# Conditions aux bords en considerant que les points aux bords ont les valeurs des points juste a cote
  #for j in range(1,N,1):
    #F_np1[0,j] =  F_np1[1,j]
  #for j in range(1,N,1):  
    #F_np1[N,j] =  F_np1[N-1,j]
  #for i in range(1,N,1):
    #F_np1[i,0] = F_np1[i,1]  
  #for i in range(1,N,1):
    #F_np1[i,N] = F_np1[i,N-1]    

  #F_np1[0,0]=F_np1[1,1]
  #F_np1[0,N]=F_np1[1,N-1]  
  #F_np1[N,0]=F_np1[N-1,1]
  #F_np1[N,N]=F_np1[N-1,N-1]
  

  
  #Les positions du temps n-1 prennent les valeurs de celles du temps n et celles du temps n prennent les valeurs du temps n+1
  F_nm1 = F_n*1.0
  F_n = F_np1*1.0

time()

1607529117.328981

In [2]:
import numpy as np # La librairie pour le calcul vectoriel, matriciel et tensoriel
import matplotlib.pyplot as plt # La librairie de plots
from time import time # La fonction qui permettra de mesurer le temps d'exécution d'une fonction
from numpy import matmul


def mul1(A, B):
  m, n = A.shape # Les dimensions de A
  _, o = B.shape # Les dimensions de B, en sachant que n_col(A) = n_row(B)
  C = np.zeros((m, o), dtype=A.dtype) # On initialise une matrice de 0 avec les dimensions attendues en faisant A x B
# Bon ça c'est la définition du produit matriciel...
  for i in range(m):
    for j in range(o):
      for k in range(n):
        C[i, j] += A[i,k]*B[k, j]
  return C

ns = 1 << np.arange(2, 9, dtype=int)
timer = np.empty((len(ns), 2), dtype="float")
for i in range(len(ns)):
  A = np.random.rand(ns[i], ns[i])
  B = np.random.rand(ns[i], ns[i])
  for f, j in zip((mul1, matmul), range(2)):
  t = time()
  C = f(A, B)
  timer[i] = time() - t

plt.semilogx(ns, timer[:, 0], label='naïve', basex=2)
plt.semilogx(ns, timer[:, 1], label='numpy', basex=2)
plt.xlabel('n')
plt.ylabel('temps (s)')
plt.title('Temps pour multiplier deux matrices nxn')
plt.legend();
plt.savefig("matrice_mul.png", dpi=1000)

IndentationError: ignored

In [14]:
# Constantes
N = 20
g = 9.8 #gravite, 9,8m/s

# Valeurs des pas
dx = 1.0/(N+1.0) 
dy = 1.0/(N+1.0)

xi = np.arange(0, 1, dx) 
yj = np.arange(0, 1, dy)



def fonc_source_nm1(p, q):
    _p = list(p)
    #_q = list(q)
    #res = _p.pop(0)
    while _p:
      #while _q:
        
        rest = np.exp(-100*((_p.pop(0)-0.5)**2+(q-0.5)**2))
        return rest

vpolyval = np.vectorize(fonc_source_nm1, excluded=['p'])

vpolyval(p=xi, q=yj)


array([1.92874985e-22, 1.79831345e-20, 1.06535888e-18, 4.01021373e-17,
       9.59135931e-16, 1.45758488e-14, 1.40743581e-13, 8.63504075e-13,
       3.36621025e-12, 8.33794709e-12, 1.31225454e-11, 1.31225454e-11,
       8.33794709e-12, 3.36621025e-12, 8.63504075e-13, 1.40743581e-13,
       1.45758488e-14, 9.59135931e-16, 4.01021373e-17, 1.06535888e-18,
       1.79831345e-20])